In [2]:
import datetime as dt
import json
import pandas as pd # type: ignore
import re
import os
import time
import sys
import logging
from google.cloud import bigquery
import numpy as np

sys.path.insert(0,os.environ["HOME"]+"/trading/python/")
from lib import aplo # type: ignore
from lib import talos_utils # type: ignore
from lib import vault_utils # type: ignore

sys.path.append(os.environ["HOME"] + "/anchorage/source/python/lib/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility # type: ignore
import utils as aql_utils # type: ignore

sys.path.append(os.environ["HOME"]+"/trading/keys/")
import keys

logging.getLogger().setLevel(logging.INFO)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.8f}'.format

In [3]:
os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"


talos_wl_api = 'ANC34AV1RAK2'
talos_wl_secret_api = 'mj9a70zpeve9rnam55kn0ql6c7sze3gw'
host_wl = "tal-160.prod.talostrading.com"

talos_api = 'ANCHJEWQR3YH'
talos_secret = '3c4nnc7lg7mfg2ae2irz5kkueeei37pz'
talos_env = "tal-42.prod.talostrading.com"

api_key_str = "4dbd027304ea4b4dd4ea6944fb61ecc633fdd4e3ec5ba21b8d7f5347ff14805c"
signing_key_str = "0b3e32e78b4d13109bda7fe5ece8616c8116ebea4cc6a8f61b5276800979073d"

aplo_key = aplo_key = "-xCvTl8CYBvFwD8MnMQsxdp4lItrun40"

api_key = 'ANC34AV1RAK2'
api_secret = 'mj9a70zpeve9rnam55kn0ql6c7sze3gw'

talos_wl_api = 'ANC34AV1RAK2'
talos_wl_secret_api = 'mj9a70zpeve9rnam55kn0ql6c7sze3gw'
#utc_now = datetime.datetime.utcnow()


In [4]:
def get_mstr_trades():
    
    client = bigquery.Client(project="production-191601")

    query = """
SELECT
    organizations.org_name AS organization_name,
    trading_client_trades.*
FROM `production-191601.sales.salesforce_user` AS salesforce_user
LEFT JOIN `production-191601.sales.salesforce_account` AS salesforce_account
    ON salesforce_account.owner_id = salesforce_user.id
FULL OUTER JOIN `brokerage.client_trades` AS trading_client_trades
    ON salesforce_account.org_id_c = trading_client_trades.organization_key_id
LEFT JOIN `client_operations.organizations_confidential` AS organizations
    ON trading_client_trades.organization_key_id = organizations.org_id
WHERE
    organizations.org_name = 'MicroStrategy'
    AND trading_client_trades.created_at >= TIMESTAMP('2023-01-01 00:00:00')
    AND trading_client_trades.created_at < CURRENT_TIMESTAMP()
    AND trading_client_trades.latest_status IN ('EXECUTED', 'SETTLED', 'SETTLING')
    AND (
        shared.ASSET_SYMBOL_MAP(trading_client_trades.symbol) NOT IN ('USDC-USD', 'USD-USDC', 'PYUSD-USD')
        OR shared.ASSET_SYMBOL_MAP(trading_client_trades.symbol) IS NULL
    )
ORDER BY
    trading_client_trades.created_at DESC;
"""

    query_job = client.query(query)
    df = query_job.result().to_dataframe()

    return df


def get_hold_trades():
    
    client = bigquery.Client(project="production-191601")

    query = """
SELECT
    organizations.org_name AS organization_name,
    trading_client_trades.*
FROM `production-191601.sales.salesforce_user` AS salesforce_user
LEFT JOIN `production-191601.sales.salesforce_account` AS salesforce_account
    ON salesforce_account.owner_id = salesforce_user.id
FULL OUTER JOIN `brokerage.client_trades` AS trading_client_trades
    ON salesforce_account.org_id_c = trading_client_trades.organization_key_id
LEFT JOIN `client_operations.organizations_confidential` AS organizations
    ON trading_client_trades.organization_key_id = organizations.org_id
WHERE
    organizations.org_name = 'Anchorage Hold LLC'
    AND trading_client_trades.created_at >= TIMESTAMP('2023-01-01 00:00:00')
    AND trading_client_trades.created_at < CURRENT_TIMESTAMP()
    AND trading_client_trades.latest_status IN ('EXECUTED', 'SETTLED', 'SETTLING')
    AND trading_client_trades.symbol IN ('BTC-USD')
    AND trading_client_trades.side IN ('SELL')
    AND (
        shared.ASSET_SYMBOL_MAP(trading_client_trades.symbol) NOT IN ('USDC-USD', 'USD-USDC', 'PYUSD-USD')
        OR shared.ASSET_SYMBOL_MAP(trading_client_trades.symbol) IS NULL
    )
ORDER BY
    trading_client_trades.created_at DESC;
"""

    query_job = client.query(query)
    df = query_job.result().to_dataframe()
   
    return df

In [5]:
mstr_trades = get_mstr_trades() 
mstr_trades['created_at'] = mstr_trades['created_at'].astype(str) 
mstr_trades['tradeDate'] = mstr_trades['created_at'].str[:10]

mstr_trades['Anch Cost Basis (BTC Price)'] = np.where(mstr_trades['fee_currency'] == 'BTC', mstr_trades['traded_quantity'] / (mstr_trades['order_quantity'] - mstr_trades['fee']), 0)
mstr_trades['Anch Cost Basis (USD Proceeds)'] = mstr_trades['Anch Cost Basis (BTC Price)'] * mstr_trades['fee']


hold_trades = get_hold_trades()
hold_trades['created_at'] = hold_trades['created_at'].astype(str) 
hold_trades['tradeDate'] = hold_trades['created_at'].str[:10]




os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "MSTR PnL"
tab_name = "MSTR_trades_new"

gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
gsu.dump_current_sheet(tab_name,mstr_trades)


tab_name = "HOLD_trades_new_dump"

gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
gsu.dump_current_sheet(tab_name,hold_trades)


/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76 UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds.

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/cloud/bigquery/table.py:1727 UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/cloud/bigquery/_pandas_helpers.py:207 UserWarning: Unable to determine type for field 'commissions'.

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76 UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds.

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/cloud/bigquery/table.py:1727 UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/cloud/bigquery/_pandas_helpers.py:207 UserWarning: Unable to determine type for field 'commissions'.

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76 UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds.

In [6]:
def multi_order_fills(all_orders,talos_env):
    combined_df = pd.DataFrame()  # Initialize an empty dataframe to store the combined data
    for identifier in all_orders:
        # Call run_function for each identifier and get the dataframe
        try:
            df = talos_env.get_trade_fills(order_id=identifier)
            df['Amount'] = df['Amount'].astype(float)
            df['Fee'] = df['Fee'].astype(float)
            df['Quantity'] = df['Quantity'].astype(float)
            df["amount_less_fees"] = df["Amount"] - df["Fee"]
            # Concatenate the dataframe to the combined dataframe
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            time.sleep(0.100)
        except Exception as e:
            err_str = f"ERROR during get_trade_fills for orderID {identifier}: {str(e)}\n\n"
            print(err_str)
    return combined_df

def fills_by_dealer(df, dealer_cutoff_start=None, dealer_cutoff_end=None):
 
    if dealer_cutoff_start and dealer_cutoff_end is not None:
        filtered_df = df.loc[(df['Timestamp'] >= dealer_cutoff_start)
                             & (df['Timestamp'] < dealer_cutoff_end)]
    else:
        filtered_df = df
    # Group by dealer
    df_by_dealer = filtered_df.groupby("Market")[["amount_less_fees", "Quantity"]].sum().reset_index()
    df_count = filtered_df.groupby("Market")["OrderID"].count().reset_index()
    df_count.rename(columns={"OrderID": "fill_count"})
    
    pd.options.display.float_format = '{:,.8f}'.format
    amount = filtered_df['amount_less_fees'].sum()
    qty = filtered_df['Quantity'].sum() 

    summary_str = f"Amount: ${amount:,.4f} \nQuantity: {qty:,.4f}\n"
    summary_df = df_by_dealer.merge(df_count)
    return filtered_df, summary_str, summary_df


from google.cloud import bigquery
import pandas as pd

bqclient = bigquery.Client(project="production-191601")

MSTR = """
SELECT
    organizations.org_name AS organization_name,
    trading_client_trades.*
FROM `production-191601.sales.salesforce_user` AS salesforce_user
LEFT JOIN `production-191601.sales.salesforce_account` AS salesforce_account
    ON salesforce_account.owner_id = salesforce_user.id
FULL OUTER JOIN `brokerage.client_trades` AS trading_client_trades
    ON salesforce_account.org_id_c = trading_client_trades.organization_key_id
LEFT JOIN `client_operations.organizations_confidential` AS organizations
    ON trading_client_trades.organization_key_id = organizations.org_id
WHERE
    organizations.org_name = 'MicroStrategy'
    AND trading_client_trades.created_at >= TIMESTAMP('2023-01-01 00:00:00')
    AND trading_client_trades.created_at < CURRENT_TIMESTAMP()
    AND trading_client_trades.latest_status IN ('EXECUTED', 'SETTLED', 'SETTLING')
    AND (
        shared.ASSET_SYMBOL_MAP(trading_client_trades.symbol) NOT IN ('USDC-USD', 'USD-USDC', 'PYUSD-USD')
        OR shared.ASSET_SYMBOL_MAP(trading_client_trades.symbol) IS NULL
    )
ORDER BY
    trading_client_trades.created_at DESC;

"""

trades_mstr = bqclient.query(MSTR).to_dataframe()

id_list = trades_mstr['id'].tolist()
filtered_dealer_trades_query = f"""
SELECT *
FROM brokerage.dealer_trades_view
WHERE symbol = 'BTC-USD'
"""

# Execute the query to filter dealer trades
filtered_dealer_trades = bqclient.query(filtered_dealer_trades_query).to_dataframe()
filtered_dealer_trades = filtered_dealer_trades[filtered_dealer_trades['parent_order_id'].isin(id_list)]

# Create a list of order_ids from the filtered_dealer_trades DataFrame
order_id_list = filtered_dealer_trades['order_id'].tolist()

# Now order_id_list contains all the order_id values
print(order_id_list)


def process_single_order(order_id, talos_env):
    results = {}  # To store the result for this single order_id

    print(f"Processing order ID: {order_id}")
        
    # Step 1: Get the combined data for the order
    combined_df = multi_order_fills([order_id], talos_env)
    print("combined_df:")
    print(combined_df)

    # Inspect column names to find where the Date/Time is stored
    print("Columns in combined_df:", combined_df.columns)

    # Step 2: Apply fills_by_dealer to get the summary
    filtered_df, summary_str, summary_df = fills_by_dealer(combined_df, dealer_cutoff_start=None, dealer_cutoff_end=None)
    
    # Step 3: Set index and add win_rate column
    summary_df = summary_df.set_index('Market')
    summary_df = summary_df.rename(columns={'OrderID': 'fill_count', 'Quantity': 'quantity'})
    summary_df.loc["TOTAL"] = summary_df.sum()  # Add a row for TOTAL
    summary_df["win_rate"] = summary_df["quantity"] / summary_df.loc["TOTAL", "quantity"]

    # Step 4: Extract the first date from the 'TransactTime' column and use the first 10 characters
    if 'TransactTime' in combined_df.columns:
        order_date = combined_df['TransactTime'].iloc[0][:10]  # Get the first 10 characters (yyyy-mm-dd)
    else:
        order_date = None

    # Step 5: Convert the dataframe into a dictionary format
    market_data_dict = summary_df.to_dict(orient="index")  # Converts each row into a dictionary keyed by Market
        
    # Store the result for this order_id, including the date
    results[order_id] = {
        "summary_str": summary_str,  # The summary string (Amount, Quantity, etc.)
        "market_data": market_data_dict,  # The market data as a dictionary
        "Date": order_date  # Add the Date of the order (yyyy-mm-dd format)
    }

    return results

# Run the function to process just one order
order_id = order_id_list[0]  # Take the first order in the list (or just define a single order)
results = process_single_order(order_id, talos_env)

# Print the results for the single order
print("Results for order ID:", order_id)
print(results[order_id])


data = []

# Iterate over each order_id and its corresponding data
for order_id, order_info in results.items():
    order_date = order_info['Date']
    market_data = order_info['market_data']

    # Iterate through each market (dealer)
    for market, market_info in market_data.items():
        # Prepare the row with the date and the quantity for each dealer
        row = {
            'Date': order_date,
            'Market': market,
            'Quantity': market_info['quantity']
        }
        data.append(row)

# Create the DataFrame
df = pd.DataFrame(data)

# Pivot the DataFrame to have Date as the index and Markets as columns
df_pivot = df.pivot(index='Date', columns='Market', values='Quantity')

# Fill any missing data with 0 (if you want it to be NaN, you can omit this step)
df_pivot = df_pivot.fillna(0)
df_pivot

def process_all_orders(order_id_list, talos_env):
    results = {}  # To store the results for all order_ids

    # Loop over each order_id in the list
    for order_id in order_id_list:
        print(f"Processing order ID: {order_id}")
        
        # Step 1: Get the combined data for the order
        combined_df = multi_order_fills([order_id], talos_env)
        print("combined_df:")
        print(combined_df)

        # Inspect column names to find where the Date/Time is stored
        print("Columns in combined_df:", combined_df.columns)

        # Step 2: Apply fills_by_dealer to get the summary
        filtered_df, summary_str, summary_df = fills_by_dealer(combined_df, dealer_cutoff_start=None, dealer_cutoff_end=None)
        
        # Step 3: Set index and add win_rate column
        summary_df = summary_df.set_index('Market')
        summary_df = summary_df.rename(columns={'OrderID': 'fill_count', 'Quantity': 'quantity'})
        summary_df.loc["TOTAL"] = summary_df.sum()  # Add a row for TOTAL
        summary_df["win_rate"] = summary_df["quantity"] / summary_df.loc["TOTAL", "quantity"]

        # Step 4: Extract the first date from the 'TransactTime' column and use the first 10 characters
        if 'TransactTime' in combined_df.columns:
            order_date = combined_df['TransactTime'].iloc[0][:10]  # Get the first 10 characters (yyyy-mm-dd)
        else:
            order_date = None

        # Step 5: Convert the dataframe into a dictionary format
        market_data_dict = summary_df.to_dict(orient="index")  # Converts each row into a dictionary keyed by Market
            
        # Store the result for this order_id, including the date
        results[order_id] = {
            "summary_str": summary_str,  # The summary string (Amount, Quantity, etc.)
            "market_data": market_data_dict,  # The market data as a dictionary
            "Date": order_date  # Add the Date of the order (yyyy-mm-dd format)
        }

    return results

# Run the function to process all order_ids
results = process_all_orders(order_id_list, talos_env)

# Print the results for each order_id
for order_id, data in results.items():
    print(f"Results for order ID {order_id}:")
    print(data)


/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76 UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds.

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/cloud/bigquery/table.py:1727 UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/cloud/bigquery/_pandas_helpers.py:207 UserWarning: Unable to determine type for field 'commissions'.

['5b90105b-ba66-4b0a-8f1e-3339b147e5a8', '497456e8-4532-481c-8763-b562953cc5e1', '5e9a15e0-6a34-43f5-b4ab-0d2e2a8f4995', '2e028e6e-2843-46d9-bfdc-ee89313bc2dd', 'b6833e4a-3fd5-4006-a24d-750657021ebf', '202a4a08-dc8b-415b-9300-f758de0fc59f', 'e4789096-67e2-4779-91b1-e55d2966751f', '7eb8a015-3438-459a-ab79-eac151992a69', '96354a4d-fc9b-4cd6-9a26-73b689f10e44', '901a3048-a4b0-4dee-91ae-c59185db1b07', '7b7ac861-acc3-4719-b147-373fb94c848b', '145dfdbe-6cff-4615-be86-f32f0ba04704', '9fa08ed9-201d-40b8-872f-65424a01396f', '74a00ae7-5b57-4a12-ba06-91b140381c57', 'f6f5b698-8932-4c7d-b88a-c369cd00e027', 'f75adc92-3b8a-4d74-bb9a-1bbe6bfaa454', '0eb3e446-d003-4178-a1bc-3cdcfc80393f', '32c26c03-8f76-4f40-a0f0-1c6be04ffc7d', '7ba513cb-ab03-494d-a4d1-a79f5b493e09', 'b7694028-260d-4ef1-88b1-b96b47f9034e', '12e608fa-0d97-4401-b89d-92e1ef105a37', 'e5e11b88-038c-4bc2-91c1-8b1ec8d189e7', '504d1af8-8449-4534-9fb2-df6a25b48ba7', '544484a3-a584-416f-afac-80258d3e7fa7', '70eaa47d-242c-487c-b94c-3c4eea55d2a5',

KeyError: 'Market'